## Project Rice price predection by ML

2023/01/13 1633i I didnt do anything yet
And I dont want to

to make this we need to use multivariate liner regression

lets get init

## import the price data as dim = 2 (with out yearly avrage)
I made the code that you dont have to change the original scv file so this code could be used from other people and its easyer for me if I have to use it later

the data tracks from 1996 bacuase that the oldest data from the rice price (kamis.or.kr)
and we tracked untill 2021 because since we have to PREDICT we are going to predic the 2022 rice price
lets see how it goes

In [2]:
import csv

data = []

#f = open("C:/Users\OWO\Documents\AA_CODE\@Projects\Projects\ai_0001_lr_rice price track\rice price data.xls")
f = open("C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/ai_0001_lr_rice price track/rice price data.csv",encoding='UTF-8')
read = csv.reader(f)

#getting data from the csv file
for a in read:
  data.append(a)
#deleting the first row because thats the legend
del data[0]

#making the price list in a dim = 1 list
pricelist = []

for year in range(0,26):
  for month in range(0,14):
    templist = []#temp list is for the structure (dimenstion) of the list since we need n,1 structued list to use for the tensor
    if (month > 0 and month < 13):
      templist.append(float(data[year][month].replace(",",'')))#since the origianl data is money it has , and its str so we have to remove "," and make it a float (it was hard)
      pricelist.append((templist))
      templist = []

f.close
#print(pricelist)
print("total data count : "+str(len(pricelist)))

total data count : 312


## import the input data

In [3]:
import csv

data = []

f = open("C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/ai_0001_lr_rice price track/statisticsDivision_20230113161645.csv",encoding='cp949')
read = csv.reader(f)

#getting data from the csv file
for a in read:
  data.append(a)
#deleting the first row because thats the legend
for i in range(0,10):#deleting 10 (unnecessary row) from it
  del data[0]

#print(data)



#list datas


##year list dim 2
year_list = []
templist = []
for i in range(0,26):
  for month_list in range(0,12):
    templist.append(1996+i)
    year_list.append(templist)
    templist = []
#print(year_list)


##month list dim 2
month_list = []
for i in range(0,26):
  for mon in range(0,12):
    templist.append(mon+1)
    month_list.append(templist)
    templist = []
#print(month_list)




##other lists
avr_temp = []
max_temp = []
min_temp = []
aor = []#it means amount of rainfall
templist = []

#print(data)
count = 0
for year in range(0,26):
  for month in range (0,12):
  
    templist.append(float(data[count][1]))
    avr_temp.append(templist)
    templist = []
    templist.append(float(data[count][2]))
    max_temp.append(templist)
    templist = []
    templist.append(float(data[count][3]))
    min_temp.append(templist)
    templist = []
    templist.append(float(data[count][4]))
    aor.append(templist)
    templist = []
    count = count + 1


## test seciton
#print(avr_temp)
#print(max_temp)
#print(min_temp)
#print(aor)

f.close

<function TextIOWrapper.close()>

## the actual ML section

In [5]:
import torch
import torch.optim as optim



elist = [1e-7,2e-7,1e-8]

pricelist = torch.FloatTensor(pricelist)
year_list = torch.FloatTensor(year_list)
month_list = torch.FloatTensor(month_list)
avr_temp = torch.FloatTensor(avr_temp)
max_temp = torch.FloatTensor(max_temp)
min_temp = torch.FloatTensor(min_temp)
aor = torch.FloatTensor(aor)



for LR in elist:
    
  w1 = torch.zeros(1, requires_grad=True)
  w2 = torch.zeros(1,requires_grad=True)
  w3 = torch.zeros(1,requires_grad=True)
  w4 = torch.zeros(1,requires_grad=True)
  w5 = torch.zeros(1,requires_grad=True)
  w6 = torch.zeros(1,requires_grad=True)
  b  = torch.zeros(1,requires_grad=True)


  optimizer = optim.SGD([w1,w2,w3,w4,w5,w6,b], lr=LR) #learning rate

  nb_epochs = 100000

  for epoch in range(nb_epochs + 1):
    #hypothesis calcuation
    hypothesis = year_list * w1 + month_list * w2 + avr_temp * w3 + max_temp * w4 + min_temp * w5 + aor * w6 + b
    
    #cost calculation
    cost = torch.mean((hypothesis - pricelist)**2)
    
    #impove H(x) with cost
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #if epoch == 100000000:
  print("LR = %.3e Epoch %d/%d w1 : %6.1f w2 : %4.1f w3 : %4.1f w4 : %4.1f w5 : %6.1f w6 : %6.1f b : %5.1f Cost : %.0f"%(LR,epoch,nb_epochs,w1.item(),w2.item(),w3.item(),w4.item(),w5.item(),w6.item(),b.item(),cost.item()))

LR = 1.000e-07 Epoch 100000/100000 w1 :   22.4 w2 : 14.3 w3 :  3.8 w4 :  3.5 w5 :    4.8 w6 :   -0.3 b :  -0.1 Cost : 22104208
LR = 2.000e-07 Epoch 100000/100000 w1 :   22.3 w2 : 25.3 w3 :  3.6 w4 :  2.8 w5 :    5.3 w6 :   -0.3 b :  -0.3 Cost : 22092028
LR = 1.000e-08 Epoch 100000/100000 w1 :   22.4 w2 :  1.7 w3 :  1.1 w4 :  1.1 w5 :    1.1 w6 :    0.3 b :  -0.0 Cost : 22126492


## Final LR

since the 2e-7 has the minimum cost we will go with that model

(i tried 1e-5 to 1e-10 btw)


## Real Test

lets test with 2022(that we didnt train with) to see how much well it could predict the price

## input 2022 price data

In [7]:
import csv

data = []

#f = open("C:/Users\OWO\Documents\AA_CODE\@Projects\Projects\ai_0001_lr_rice price track\rice price data.xls")
f = open("C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/ai_0001_lr_rice price track/rice price data.csv",encoding='UTF-8')
read = csv.reader(f)

#getting data from the csv file
for a in read:
  data.append(a)
#deleting the first row because thats the legend
del data[0]

#making the price list in a dim = 1 list
predict_pricelist = []

for year in range(26,27):
  for month in range(0,14):
    templist = []#temp list is for the structure (dimenstion) of the list since we need n,1 structued list to use for the tensor
    if (month > 0 and month < 13):
      templist.append(float(data[year][month].replace(",",'')))#since the origianl data is money it has , and its str so we have to remove "," and make it a float (it was hard)
      predict_pricelist.append((templist))
      templist = []

f.close
print(predict_pricelist)
print("total data count : "+str(len(predict_pricelist)))

[[53450.0], [52597.0], [52459.0], [52037.0], [51221.0], [50990.0], [50643.0], [49228.0], [48397.0], [49806.0], [53284.0], [51900.0]]
total data count : 12


## input 2022 input data

In [11]:
import csv

data = []

f = open("C:/Users/OWO/Documents/AA_CODE/@Projects/Projects/ai_0001_lr_rice price track/statisticsDivision_20230113161645.csv",encoding='cp949')
read = csv.reader(f)

#getting data from the csv file
for a in read:
  data.append(a)
#deleting the first row because thats the legend
for i in range(0,10):#deleting 10 (unnecessary row) from it
  del data[0]

#print(data)



#list datas


##year list dim 2
predict_year_list = []
templist = []
for i in range(26,27):
  for month in range(0,12):
    templist.append(1996+i)
    predict_year_list.append(templist)
    templist = []
print(predict_year_list)


##month list dim 2
predict_month_list = []
for i in range(26,27):
  for mon in range(0,12):
    templist.append(mon+1)
    predict_month_list.append(templist)
    templist = []
print(predict_month_list)




##other lists
predict_avr_temp = []
predict_max_temp = []
predict_min_temp = []
predict_aor = []#it means amount of rainfall
templist = []

#print(data)
count = 312#its 322 but we deleted 10 on the top so... its num - 10
for year in range(26,27):
  for month in range (0,12):
  
    templist.append(float(data[count][1]))
    predict_avr_temp.append(templist)
    templist = []
    templist.append(float(data[count][2]))
    predict_max_temp.append(templist)
    templist = []
    templist.append(float(data[count][3]))
    predict_min_temp.append(templist)
    templist = []
    templist.append(float(data[count][4]))
    predict_aor.append(templist)
    templist = []
    count = count + 1


## test seciton
print(predict_avr_temp)
print(predict_max_temp)
print(predict_min_temp)
print(predict_aor)

f.close

[[2022], [2022], [2022], [2022], [2022], [2022], [2022], [2022], [2022], [2022], [2022], [2022]]
[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12]]
[[-0.3], [0.3], [7.9], [13.8], [18.0], [22.3], [26.0], [25.4], [21.1], [14.2], [10.0], [-0.7]]
[[5.3], [5.9], [13.8], [20.4], [24.6], [27.1], [30.3], [29.3], [26.4], [20.2], [16.7], [4.3]]
[[-5.3], [-4.9], [2.3], [7.7], [11.7], [18.4], [22.6], [22.3], [16.9], [9.3], [4.4], [-5.2]]
[[4.8], [4.7], [92.0], [60.5], [6.1], [180.7], [184.3], [280.8], [157.8], [76.7], [62.3], [17.2]]


<function TextIOWrapper.close()>

In [12]:
## actual test

w1 =   22.3
w2 = 25.3
w3 =  3.6
w4 =  2.8
w5 =    5.3
w6 =   -0.3
b =  -0.3

predict_cost_sum = 0

for i in range(0,12):
  predict_cost_sum = (predict_year_list[i][0] * w1 + predict_month_list[i][0] * w2 + predict_avr_temp[i][0] * w3 + w4 * predict_max_temp[i][0] + w5 * predict_min_temp[i][0] + w6 * predict_aor[i][0] + b)-predict_pricelist[i][0]
print(predict_cost_sum/12)

-544.1083333333336


### the END CODE

In [10]:
#data
w1 =   22.3
w2 = 25.3
w3 =  3.6
w4 =  2.8
w5 =    5.3
w6 =   -0.3
b =  -0.3


print("hello this is the Korean Rice Price Predictor")
print("type in the data then you will recive the price by its data")

new_year = float(input("type in the year : "))
new_month = float(input("type in the month : "))
new_avr_temp = float(input("type in the average temperature of that month : "))
new_max_temp = float(input("type in the maximum temperature of that month : "))
new_min_temp = float(input("input the minimum temperture of that month : "))
new_aor = float(input("input the amount of rain in that month : "))

print(" the AI is guessing it would be : ",end="")
print("%.0f Won"%(new_year * w1 + new_month * w2 + new_avr_temp * w3 + new_max_temp * w4 + new_min_temp * w5 + new_aor * w6 + b)
)

hello this is the Korean Rice Price Predictor
type in the data then you will recive the price by its data
 the AI is guessing it would be : 45100 Won


## After the project

it's 2023/01/14 

I think this took about 4 hours to code
it was not very hard but it was a little bit tricky to get the CSV file to list and use that

# improvements needed

I think I missed a lot of things. I need to research before I choose what data to learn from because I'm not sure if the month is going to make a big effect on the price (I thought so but I think It doesn't at the end) and about rice, you harvest them only once a year (at least in Korea) so I don't it look like the temperature of the month had a lot of effect on the price.
And also an important thing is that Most of the time, the price is too cheap than expected. I think it's becuase the data was too varied about the date. Since the economy of Korea grew so fast and rice prices grew fast with it, it couldn't catch up with the new prices, so in this case I should have been tracking (or learning) the last 5 years of data, which (I think) it could have improved the AI

# about the learning rate

I made a new system for the learning rate. In this one, the learning rate is made on list so you can put many learning rate and print the cost in every learning rate to see what is the most optimized learning rate
And I keep getting nan by change the learing rate. I dont know why at the moment. I have to study on that.

# end
Im tired
Im going to bed